# utf8编码

In [3]:
"中国".encode('utf-8')

b'\xe4\xb8\xad\xe5\x9b\xbd'

In [3]:
print(list("中国".encode('utf-8')))
print(list("hello".encode('utf-8')))
print(["中国".encode('utf-8')])

[228, 184, 173, 229, 155, 189]
[104, 101, 108, 108, 111]
[b'\xe4\xb8\xad\xe5\x9b\xbd']


In [6]:
text = " 中国，是一个拥有五千年悠久历史和灿烂文明的国家。中国地大物博，地域辽阔，从北方的冰雪边疆到南方的热带雨林，展现出丰富多样的自然风貌。中国不仅是世界四大文明古国之一，也是当今全球第二大经济体，拥有强大的制造能力和科技创新实力。中国文化源远流长，汉字、京剧、书法、美食等都深受世界喜爱。如今，中国正以更加自信、开放的姿态走向世界舞台的中央，展现出属于中国的魅力与力量。"
tokeniized_text = (list(text.encode('utf-8')))

In [ ]:
pattern = [228, 184, 173, 229, 155, 189]
count = 0
for i in range(len(tokeniized_text)):
    if(pattern[0]== tokeniized_text[i] and pattern[1]==tokeniized_text[i+1] and pattern[2]==tokeniized_text[i+2] and pattern[3]==tokeniized_text[i+3] and pattern[4]==tokeniized_text[i+4] and pattern[5]==tokeniized_text[i+5]):
        count += 1
print(count)

6


# momo的tokenizer

## BPE代码

In [17]:
import re, collections

text = "The aims for this subject is for students to develop an understanding of the main algorithms used in naturallanguage processing, for use in a diverse range of applications including text classification, machine translation, and question answering. Topics to be covered include part-of-speech tagging, n-gram language modelling, syntactic parsing and deep learning. The programming language used is Python, see for more information on its use in the workshops, assignments and installation at home."

# 统计词频
def get_vocab(text):
    vocab = collections.defaultdict(int)

    for word in text.strip().split():
        vocab[' '.join(list(word))+ '</w>'] += 1
    return vocab

print(get_vocab(text))

defaultdict(<class 'int'>, {'T h e</w>': 2, 'a i m s</w>': 1, 'f o r</w>': 4, 't h i s</w>': 1, 's u b j e c t</w>': 1, 'i s</w>': 2, 's t u d e n t s</w>': 1, 't o</w>': 2, 'd e v e l o p</w>': 1, 'a n</w>': 1, 'u n d e r s t a n d i n g</w>': 1, 'o f</w>': 2, 't h e</w>': 2, 'm a i n</w>': 1, 'a l g o r i t h m s</w>': 1, 'u s e d</w>': 2, 'i n</w>': 3, 'n a t u r a l l a n g u a g e</w>': 1, 'p r o c e s s i n g ,</w>': 1, 'u s e</w>': 2, 'a</w>': 1, 'd i v e r s e</w>': 1, 'r a n g e</w>': 1, 'a p p l i c a t i o n s</w>': 1, 'i n c l u d i n g</w>': 1, 't e x t</w>': 1, 'c l a s s i f i c a t i o n ,</w>': 1, 'm a c h i n e</w>': 1, 't r a n s l a t i o n ,</w>': 1, 'a n d</w>': 3, 'q u e s t i o n</w>': 1, 'a n s w e r i n g .</w>': 1, 'T o p i c s</w>': 1, 'b e</w>': 1, 'c o v e r e d</w>': 1, 'i n c l u d e</w>': 1, 'p a r t - o f - s p e e c h</w>': 1, 't a g g i n g ,</w>': 1, 'n - g r a m</w>': 1, 'l a n g u a g e</w>': 2, 'm o d e l l i n g ,</w>': 1, 's y n t a c t i c</w>

In [19]:
def get_stats(vocab):
    pairs = collections.defaultdict(int)
    for word, freq in vocab.items():
        symbols = word.split()
        for i in range(len(symbols)-1):
            pairs[(symbols[i], symbols[i+1])] += freq
    return pairs

In [20]:
"""
EXAMPLE:
    word = 'T h e <\w>'
    pair = ('e', '<\w>')
    word_after_merge = 'T h e<\w>'    
输入:
    pair: Tuple[str, str] # 需要合并的字符对
    v_in: Dict[str, int]  # 合并前的vocab   
输出:
    v_out: Dict[str, int] # 合并后的vocab    
注意:
    当合并word 'Th e<\w>'中的字符对 ('h', 'e')时，'Th'和'e<\w>'字符对不能被合并。
"""
def merge_vocab(pair, v_in):
    v_out = {}
    # 把pair拆开，然后用空格合并起来，然后用\把空格转义
    bigram = re.escape(' '.join(pair))
    # 自定义一个正则规则, (?<!\S)h\ e(?!\S) 只有前面、后面不是非空白字符(\S)(意思前后得是没东西的)，才匹配h\ e，这样就可以把Th\ e<\w>排除在外
    p = re.compile(r'(?<!\S)' + bigram + r'(?!\S)')
    
    for v in v_in:
        # 遍历当前的vocabulary，找到匹配正则的v时，才用合并的pair去替换变成新的pair new，如果没有匹配上，那就保持原来的。
        # 比如pair当前是'h'和'e'，然后遍历vocabulary，找到符合前后都没有东西只有'h\ e'的时候就把他们并在一起变成'he'
        new = p.sub(''.join(pair),v)
        # 然后新的合并的数量就是当前vocabulary里面pair对应的数量
        v_out[new] = v_in[v]
    return v_out